pip install xlrd

In [1]:
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# Extract
## Yahoo Finance

In [2]:
# get s&p 500, nasdaq, dow jones, nikkei, eurostoxx, ftse 100 closing prices & volume

# List of tickers for the indices
tickers = {
    'SP500': '^GSPC',
    'NASDAQ': '^IXIC',
    'DJ': '^DJI',
    'Nikkei': '^N225',
    'Stoxx': '^STOXX50E',
    'FTSE': '^FTSE',
    'Gold': 'GC=F',
    'Silver': 'SI=F',
    'Oil': 'CL=F',
    'Gas': 'NG=F',
    'EUR/USD': 'EURUSD=X',
    'USD/JPY': 'JPY=X',
    'GBP/USD': 'GBPUSD=X',
    'US_10Y': '^TNX',
    'US_2Y': '^IRX',
    'US Corporate Bonds': 'LQD',  # iShares iBoxx $ Investment Grade Corporate Bond ETF
    'US HY Bonds': 'HYG',  # iShares iBoxx $ High Yield Corporate Bond ETF
    'VIX': '^VIX'
}

# Dictionary to store the data
data = {}

for name, ticker in tickers.items():
    index_data = yf.Ticker(ticker).history(period="5y", interval="1d")[["Close", "Volume"]]
    index_data.index = index_data.index.strftime('%Y-%m-%d')
    data[name+'_Close'] = index_data['Close']
    if name not in ['EUR/USD', 'USD/JPY', 'GBP/USD', 'US_10Y', 'US_2Y', 'VIX']:
        data[name+'_Volume'] = index_data['Volume']
    data[name+'_Returns'] = index_data['Close'].pct_change()   #très corrélé avec les log returns
    #data[name+'_Log_Returns'] = np.log(data[name+'_Close']/data[name+'_Close'].shift(1))
    data[name+'_Volatility_20d'] = data[name+'_Returns'].rolling(window=20).std() * np.sqrt(252)


# Convert to DataFrame
df = pd.concat(data.values(), keys=data.keys(), axis=1)
df.index = pd.to_datetime(df.index)

## US interest rates (FRED)

In [3]:
# get interest rates from FRED for the last 5 years
end = datetime.datetime.now()
start = end - datetime.timedelta(days=5*365)

dict_maturities={}
#maturities = [1/12, 0.25, 0.5,1,2,3,5,7,10,20,30]
maturities = [1/12] # très grande corrélation entre les taux donc on en garde qu'un
for i in maturities:
    if(i<1):
        dict_maturities[str(int(i*12))+'M']='DGS'+str(int(i*12))+'MO'
    else:
        dict_maturities[str(i)+'Y']='DGS'+str(i)

data_ir = pd.DataFrame()
for key, series_id in dict_maturities.items():
    data_ir[key] = pdr.get_data_fred(series_id, start, end)
data_ir = data_ir.dropna()/100    #we delete the dates with missing values and convert to percentage

# add CPI data
data_cpi = pdr.get_data_fred('CPIAUCSL', start, end)
data_ir['CPI'] = data_cpi

data_ir.index = pd.to_datetime(data_ir.index)

## Geopolitical events

source : https://www.matteoiacoviello.com/gpr.htm, explanations are in the excel file after the Ith column

| **var_name**   | **var_label**                                  |
|----------------|-----------------------------------------------|
| DAY            | DAY                                           |
| N10D           | Number of articles (10 recent newspapers, 1985-) |
| GPRD           | Daily GPR (Index: 1985:2019=100)              |
| GPRD_ACT       | Daily GPR Acts (Index: 1985:2019=100)         |
| GPRD_THREAT    | Daily GPR Threats (Index: 1985:2019=100)      |
| date           | Date                                          |
| GPRD_MA30      | 30 day moving average of Daily GPR            |
| GPRD_MA7       | 7 day moving average of Daily GPR             |
| event          | Major event label                             |


In [4]:
geopol_events = pd.read_excel("data_gpr_daily_recent.xls", sheet_name="Sheet1", usecols="A:I")
geopol_events = geopol_events.drop(columns=["DAY"])
geopol_events.columns = ["N10D", "GPRD", "GPRD_ACT", "GPRD_THREAT", "DATE", "GPRD_MA30", "GPRD_MA7", "EVENT"]
geopol_events.index = pd.to_datetime(geopol_events["DATE"])
geopol_events = geopol_events.drop(columns=["DATE"])

## Options volume

In [5]:
# Source : https://www.cboe.com/us/options/market_statistics/historical_data/
df_vol_options = pd.read_csv("daily_volume_SPX_2019-01-01_2025-01-27.csv", index_col=0)
df_vol_options.index = pd.to_datetime(df_vol_options.index)
df_vol_options = df_vol_options["Volume"]
df_vol_options.name = "Volume_Options_SPX"

## Merging datasets

In [6]:
df = df.merge(data_ir, left_index=True, right_index=True, how="left")
df = df.merge(geopol_events, left_index=True, right_index=True, how="left")
df = df.merge(df_vol_options, left_index=True, right_index=True, how="left")


## Other
J'aurais voulu ajouter les earnings reports dates mais j'ai pas trouvé de dataset.

# Transform

In [7]:
df.sort_index(inplace=True)

In [8]:
df["CPI"] = df["CPI"].ffill() # CPI is monthly so we keep constant value for the month
df["1M"] = df["1M"].ffill() # if there is no value for the day we keep the last value
#df["EVENT"] = df["EVENT"].fillna("None") # if there is no event we put None
df.loc[~df["EVENT"].isna(), "EVENT"] = 1    # replace the NaN values in the geopolitical events by 0 and the others by 1
df["EVENT"] = df["EVENT"].fillna(0)
df[["Nikkei_Close", "Nikkei_Volume", "Nikkei_Returns", "Nikkei_Volatility_20d", "Stoxx_Close", "Stoxx_Volume", "Stoxx_Returns", "Stoxx_Volatility_20d", "FTSE_Close", "FTSE_Volume", "FTSE_Returns", "FTSE_Volatility_20d"]] = df[["Nikkei_Close", "Nikkei_Volume", "Nikkei_Returns", "Nikkei_Volatility_20d", "Stoxx_Close", "Stoxx_Volume", "Stoxx_Returns", "Stoxx_Volatility_20d", "FTSE_Close", "FTSE_Volume", "FTSE_Returns", "FTSE_Volatility_20d"]].ffill() # if foreign markets are closed, we keep the last value

df = df.loc[~df["SP500_Close"].isna()] # we don't try to predict volatility when the market is closed

df = df[22:-2]  # we remove the first days because we need 20 days to calculate the volatility and the last days because data may be not available because it is too recent

/tmp/ipykernel_18874/2905507047.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["EVENT"] = df["EVENT"].fillna(0)


In [9]:
df

,SP500_Close,SP500_Volume,SP500_Returns,SP500_Volatility_20d,NASDAQ_Close,NASDAQ_Volume,NASDAQ_Returns,NASDAQ_Volatility_20d,DJ_Close,DJ_Volume,...,1M,CPI,N10D,GPRD,GPRD_ACT,GPRD_THREAT,GPRD_MA30,GPRD_MA7,EVENT,Volume_Options_SPX
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-02,3090.229980,6.381330e+09,0.046039,0.300728,8952.169922,4.249020e+09,0.044915,0.318457,26703.320312,6.372000e+08,...,0.0141,NaN,686.0,106.233948,125.162766,104.398109,75.564842,83.912109,0,2289135.0
2020-03-03,3003.370117,6.376510e+09,-0.028108,0.312317,8684.089844,4.336700e+09,-0.029946,0.329618,25917.410156,6.470800e+08,...,0.0111,NaN,700.0,123.038231,141.530197,86.570122,78.294449,84.995941,0,1582850.0
2020-03-04,3130.120117,5.073020e+09,0.042203,0.346980,9018.089844,3.634760e+09,0.038461,0.351640,27090.859375,4.575900e+08,...,0.0100,NaN,800.0,144.924835,140.350784,185.929016,79.541359,91.042107,0,2085874.0
2020-03-05,3023.939941,5.579290e+09,-0.033922,0.360247,8738.589844,3.763860e+09,-0.030993,0.365051,26121.279297,4.773700e+08,...,0.0092,NaN,753.0,79.184891,122.797340,58.528637,77.393906,93.216118,0,1511953.0
2020-03-06,2972.370117,6.555240e+09,-0.017054,0.361535,8575.620117,4.292730e+09,-0.018649,0.366330,25864.779297,5.997800e+08,...,0.0079,NaN,839.0,51.327030,55.105122,78.793915,76.826561,92.671585,0,2446350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-21,6049.240234,4.702920e+09,0.008768,0.144783,19756.779297,8.015780e+09,0.006448,0.195159,44025.808594,5.922100e+08,...,0.0442,316.441,480.0,55.209465,27.519762,80.339691,115.908852,107.304070,0,3321643.0
2025-01-22,6086.370117,4.323040e+09,0.006138,0.145382,20009.339844,7.219060e+09,0.012783,0.199367,44156.730469,5.884300e+08,...,0.0442,316.441,511.0,175.028030,193.876984,161.712555,116.868111,107.205818,0,3629822.0
2025-01-23,6118.709961,4.432250e+09,0.005313,0.142125,20053.679688,6.837700e+09,0.002216,0.196777,44565.070312,4.441800e+08,...,0.0445,316.441,497.0,133.302521,132.892212,110.845230,116.521667,113.483841,0,3178667.0


In [10]:
s=0
for x in df.columns:
    a = df[x].isna().sum()
    if a>0:
        print(x, a)
    s+=a

if s==0:
    print("No missing values")

CPI 22


Il manque que les données de début de période du CPI.

## Feature engineering

Market sentiment analysis: to be done

In [ ]:
# Relative strength index
def rsi(series, period):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
    
for name, ticker in tickers.items():
    ema_12 = df[name+'_Close'].ewm(span=12, adjust=False).mean()
    df[name+'_EMA26'] = df[name+'_Close'].ewm(span=26, adjust=False).mean()
    df[name+'_MACD'] = ema_12 - df[name+'_EMA26']
    df[name+'_Signal'] = df[name+'_MACD'].ewm(span=9, adjust=False).mean()

    df[name+'_RSI10'] = rsi(df[name+'_Close'], 10)
    df[name+'_RSI22'] = rsi(df[name+'_Close'], 22)

In [12]:
df_corr = df.corr()

In [ ]:
# Create the heatmap
fig = px.imshow(
    df_corr,
    text_auto=False,  # Display correlation values on the heatmap
    color_continuous_scale='RdYlBu_r',  # Choose a color scale
    labels=dict(x="Features", y="Features", color="Correlation"),
    title="Correlation Heatmap"
)

# Customize layout
fig.update_layout(
    xaxis_title="Features",
    yaxis_title="Features",
    width=1500,
    height=1500,
    xaxis=dict(tickfont=dict(size=10)),  # Reduce x-axis label size
    yaxis=dict(tickfont=dict(size=10))   # Reduce y-axis label size
)

# Show the plot
fig.show()